In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from statistics import mean
from scipy import stats

In [ ]:
results_folder = os.getcwd()
csv_details = "EXPERIMENTS.csv"
csv_explicit = "explicit.csv"
csv_implicit = "implicit.csv"
csv_init = "init.csv"

folder_plots = "plots/godspeed/"

In [ ]:
exp_details = pd.read_csv(results_folder+"/"+csv_details)
survey_explicit = pd.read_csv(results_folder+"/"+csv_explicit)
survey_implicit = pd.read_csv(results_folder+"/"+csv_implicit)

# Isolate columns that we want
# In surveys, Godspeed is from column 27 (starting from 0).
for i in range(27):
    survey_implicit.drop(survey_implicit.columns[[0]], axis=1, inplace=True)
    survey_explicit.drop(survey_explicit.columns[[0]], axis=1, inplace=True)

In [ ]:
survey_explicit

In [ ]:
sections = ["Anthropomorphism", "Animacy", "Likeability", "Intelligence", "Safety",
           "OVERALL", "SYSUSE", "INFOQUAL", "INTERQUAL"]

sections_upper = [section.lower().upper() for section in sections]

sections_plot = ["Anthropomorphism", "Animacy", "Likeability", "Perceived\nIntelligence", "Perceived\nSafety",
           "Overall", "System\nUsefulness", "Information\nQuality", "Interface\nQuality"]

# CREATE GODSPEED CSV

In [ ]:
def get_mean_godspeed(df_survey, from_col, to_col):
    mean_godspeed = []
    for index in range(len(df_survey)):
        all_values = df_survey.iloc[index].tolist()
        godspeed_index = all_values[from_col:to_col] # 5, 6 or 3 items depending on section
        #print(len(godspeed_index))
        mean_godspeed.append(np.round(mean(godspeed_index), 2))
    return mean_godspeed

OTHER METRICS:

OVERALL: Average responses to 1 - 19

System Usefulness (SYSUSE): average responses to 1 - 8

Information Quality (INFOQUAL): average responses to 9 - 15

Interface Quality (INTERQUAL): average responses to 16 - 18

In [ ]:
godspeed_implicit = {}
godspeed_explicit = {}
# Anthropomorphism
godspeed_implicit["anthropomorphism"] = get_mean_godspeed(survey_implicit, 0, 5)
godspeed_explicit["anthropomorphism"] = get_mean_godspeed(survey_explicit, 0, 5)

#Animacy
godspeed_implicit["animacy"] = get_mean_godspeed(survey_implicit, 5, 11)
godspeed_explicit["animacy"] = get_mean_godspeed(survey_explicit, 5, 11)

#Likeability
godspeed_implicit["likeability"] = get_mean_godspeed(survey_implicit, 11, 16)
godspeed_explicit["likeability"] = get_mean_godspeed(survey_explicit, 11, 16)

#Perceived intelligence
godspeed_implicit["intelligence"] = get_mean_godspeed(survey_implicit, 16, 21)
godspeed_explicit["intelligence"] = get_mean_godspeed(survey_explicit, 16, 21)

#Perceived safety
godspeed_implicit["safety"] = get_mean_godspeed(survey_implicit, 21, 24)
godspeed_explicit["safety"] = get_mean_godspeed(survey_explicit, 21, 24)

# OVERALL: Average responses to 1 - 19 [0-18]
godspeed_implicit["overall"] = get_mean_godspeed(survey_implicit, 0, 19)
godspeed_explicit["overall"] = get_mean_godspeed(survey_explicit, 0, 19)

# System Usefulness (SYSUSE): average responses to 1 - 8 [0-7]
godspeed_implicit["sysuse"] = get_mean_godspeed(survey_implicit, 0, 8)
godspeed_explicit["sysuse"] = get_mean_godspeed(survey_explicit, 0, 8)

# Information Quality (INFOQUAL): average responses to 9 - 15 [8-14]
godspeed_implicit["infoqual"] = get_mean_godspeed(survey_implicit, 8, 15)
godspeed_explicit["infoqual"] = get_mean_godspeed(survey_explicit, 8, 15)

# Interface Quality (INTERQUAL): average responses to 16 - 18 [15-17]
godspeed_implicit["interqual"] = get_mean_godspeed(survey_implicit, 15, 18)
godspeed_explicit["interqual"] = get_mean_godspeed(survey_explicit, 15, 18)


### If we want to consider the order of questionnaires submission

In [ ]:
# Create list of first and second surveys (explicit, implicit)
'''
first_surveys = []
second_surveys = []
for index in np.unique(exp_details['ID']):
    order_behaviors = exp_details[exp_details['ID'] == index]['BEHAVIOR'].tolist()
    first_surveys.append(order_behaviors[0])
    second_surveys.append(order_behaviors[1])

# Create dataframe
df = pd.DataFrame(np.unique(exp_details['ID']), columns = ['ID'] )
df['FIRST_SURVEY'] = first_surveys
df['SECOND_SURVEY'] = second_surveys

# ADD sections
for section in sections:
    first_values = []
    second_values = []
    for i in range(len(first_surveys)):
        if first_surveys[i] == 'explicit' and second_surveys[i] == 'implicit':
            first_values.append(godspeed_explicit[section][i])
            second_values.append(godspeed_implicit[section][i])
        elif first_surveys[i] == 'implicit' and second_surveys[i] == 'explicit':
            first_values.append(godspeed_implicit[section][i])
            second_values.append(godspeed_explicit[section][i])
        else:
            print("ERROR: in explicit or implicit trust sequence.")
    df['FIRST_'+section.upper()] = first_values
    df['SECOND_'+section.upper()] = second_values
    
# Save csv
#df.to_csv("results_godspeed.csv", index=False)
'''

### If we want to consider only type of questionnaire referring to the behavior (explicit, implicit)

In [ ]:
# Create dataframe
df = pd.DataFrame(np.unique(exp_details['ID']), columns = ['ID'] )

# ADD sections
for section in sections:
    section = section.lower()
    df['E_'+section.upper()] = godspeed_explicit[section]
    df['I_'+section.upper()] = godspeed_implicit[section]

# Save csv
df.to_csv("results_godspeed.csv", index=False)

In [ ]:
# Create dataframe
index = 0

df1 = pd.DataFrame(np.concatenate((np.unique(exp_details['ID']),np.unique(exp_details['ID']))), columns = ['ID'])
section = sections_upper[index].lower()
df1["values"] = godspeed_explicit[section]+godspeed_implicit[section]
df1["section"] = [sections_plot[index] for i in range(2*len(godspeed_explicit[section]))]
df1['Behavior'] = ['explicit' for i in range(len(godspeed_explicit[section]))]+['implicit' for i in range(len(godspeed_explicit[section]))]
index += 1
    
while index < len(sections):
    # ADD sections
    df2 = pd.DataFrame(np.concatenate((np.unique(exp_details['ID']),np.unique(exp_details['ID']))), columns = ['ID'])
    section = sections_upper[index].lower()
    df2["values"] = godspeed_explicit[section]+godspeed_implicit[section]
    df2["section"] = [sections[index] for i in range(2*len(godspeed_explicit[section]))]
    df2['Behavior'] = ['explicit' for i in range(len(godspeed_explicit[section]))]+['implicit' for i in range(len(godspeed_explicit[section]))]
    index += 1

    df1 = pd.concat([df1, df2], axis=0)
        
# Save csv
df1.to_csv("results_godspeed_2.csv", index=False)

# ANALYSIS

In [ ]:
df = pd.read_csv("results_godspeed.csv")
#df.drop(df[df['ID'] == 59].index, inplace=True)
df

In [ ]:
statistics = pd.DataFrame(sections, columns = ['SECTION'] )

In [ ]:
def update_statistics(df, indexes, behavior, title):
    for section in sections:
        if behavior == "explicit":
            if indexes:
                col = df['E_'+section.upper()][indexes].tolist()
            else:
                col = df['E_'+section.upper()].tolist()
        else:
            if indexes:
                col = df['I_'+section.upper()][indexes].tolist()
            else:
                col = df['I_'+section.upper()].tolist()
            
        row = statistics[statistics['SECTION'] == section].index
        statistics.loc[row, 'MEAN_'+title] = np.round(mean(col), 2)
        statistics.loc[row, 'STD_'+title] = np.round(np.std(col), 2)

        #statistic, pvalue = stats.kstest(col_1, col_2)
        #statistics.loc[row, 'PVALUE'] = np.round(pvalue, 2)
    

color_true_pred = "#2ca02c"
color_younger_pred = "black"
color_older_pred = "r"

In [ ]:
update_statistics(df, indexes=None, behavior="explicit", title="EXPLICIT")
update_statistics(df, indexes=None, behavior="implicit", title="IMPLICIT")
statistics

In [ ]:
statistics["MEAN_EXPLICIT"] - statistics["MEAN_IMPLICIT"]

### EXPLICIT

In [ ]:
numerosity = {}

In [ ]:
order_ascending = True

In [ ]:
id_true_pred = [0, 1, 4, 5, 6, 8, 9, 10, 11, 13, 14, 16, 17, 18, 19, 20, 23, 24, 25, 26, 28, 31, 33, 35, 36, 37, 40, 41, 42, 43, 44, 46, 47, 49, 50, 51, 54, 57]
id_younger_pred = [29, 34, 39, 45, 48, 52, 53, 55, 56, 59, 60, 61]
id_older_pred = [2, 3, 7, 12, 15, 21, 22, 27, 30, 32, 38, 58]

update_statistics(df, id_true_pred, "explicit", "TRUE")
update_statistics(df, id_younger_pred, "explicit", "YOUNGER")
update_statistics(df, id_older_pred, "explicit", "OLDER")
statistics

In [ ]:
numerosity['E_true'] = len(id_true_pred)
numerosity['E_younger'] = len(id_younger_pred)
numerosity['E_older'] = len(id_older_pred)

In [ ]:
if order_ascending:
    statistics.sort_values("MEAN_TRUE", inplace=True)
    x_values = statistics["SECTION"]
else:
    x_values = sections_plot

In [ ]:
# Mean of Godspeed sections based age group and gender prediction in the EXPLICIT case
fig, ax = plt.subplots(1,1,figsize=(14, 5)) 

true = statistics["MEAN_TRUE"].tolist()
ax.plot(x_values, true, 'o--', color=color_true_pred, label="Correct age group and gender") 

younger = statistics["MEAN_YOUNGER"].tolist()
ax.plot(x_values, younger, 'o--', color=color_younger_pred, label="Younger age group and correct gender") 

older = statistics["MEAN_OLDER"].tolist()
ax.plot(x_values, older, 'o--', color=color_older_pred, label="Older age group or wrong gender") 

plt.yticks(np.arange(0, 5.5, 0.5))
plt.grid(axis='x')
plt.ylabel('Mean')
plt.legend(loc='lower left')
plt.xlabel('Section')
plt.title('Godspeed mean in explicit behavior')

#plt.savefig(folder_plots+"/explicit.jpg", bbox_inches='tight')
plt.show()

### IMPLICIT

In [ ]:
statistics = pd.DataFrame(statistics["SECTION"], columns = ['SECTION'] )

In [ ]:
id_true_pred = [3, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16, 17, 19, 20, 23, 24, 25, 28, 30, 31, 33, 37, 38, 40, 41, 42, 43, 44, 46, 47, 50, 51, 54, 58, 60]
id_younger_pred = [0, 18, 26, 29, 34, 36, 39, 45, 48, 52, 53, 55, 56, 59, 61]
id_older_pred = [1, 2, 7, 11, 14, 21, 22, 27, 32, 35, 49, 57]

update_statistics(df, id_true_pred, "implicit", "TRUE")
update_statistics(df, id_younger_pred, "implicit", "YOUNGER")
update_statistics(df, id_older_pred, "implicit", "OLDER")
statistics

In [ ]:
numerosity['I_true'] = len(id_true_pred)
numerosity['I_younger'] = len(id_younger_pred)
numerosity['I_older'] = len(id_older_pred)

In [ ]:
if order_ascending:
    x_values = statistics["SECTION"]
else:
    x_values = sections_plot

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(14, 5)) 

true = statistics["MEAN_TRUE"].tolist()
ax.plot(x_values, true, 'o--', color=color_true_pred, label="Correct age group and gender") 

younger = statistics["MEAN_YOUNGER"].tolist()
ax.plot(x_values, younger, 'o--', color=color_younger_pred, label="Younger age group and correct gender") 

older = statistics["MEAN_OLDER"].tolist()
ax.plot(x_values, older, 'o--', color=color_older_pred, label="Older age group or wrong gender") 


plt.yticks(np.arange(0, 5.5, 0.5))
plt.grid(axis='x')
plt.ylabel('Mean')
plt.legend(loc='lower left')
plt.xlabel('Section')
plt.title('Godspeed mean in implicit behavior')

#plt.savefig(folder_plots+"/implicit.jpg", bbox_inches='tight')
plt.show()

In [ ]:
labels = ["Correct age group and\ngender", "Younger age group and\ncorrect gender", "Older age group or\nwrong gender"]
values_explicit = list(numerosity.values())[0:3]
values_implicit = list(numerosity.values())[3:6]
  
fig, ax = plt.subplots(1,1,figsize=(10, 5)) 

# Set position of bar on X axis
barWidth = 0.25
br1 = np.arange(len(values_explicit))
br2 = [x + barWidth for x in br1]


plt.bar(br1, values_explicit, width = barWidth, edgecolor ='grey', label ='Explicit')
plt.bar(br2, values_implicit, width = barWidth, edgecolor ='grey', label ='Implicit')

ax.set_axisbelow(True)
plt.grid(axis='y')

plt.ylabel("Number of predictions")
plt.xticks([r + barWidth/2 for r in range(len(labels))], labels)
plt.yticks(np.arange(0, 40, 2))
plt.xlabel("Prediction outcome")
plt.title("Numerosity of prediction outcomes")

plt.legend()
plt.savefig(folder_plots+"/numerosity.jpg", bbox_inches='tight')
plt.show()

In [ ]:
np.sum(list(numerosity.values()))

# STATISTICS

In [ ]:
df = pd.read_csv("results_godspeed.csv")
df

In [ ]:
statistics = pd.DataFrame(sections_upper, columns = ['SECTION'] )
for section in sections_upper:
    data_e = df["E_"+section].tolist()
    data_i = df["I_"+section].tolist()
    
    statistics_e = []
    statistics_i = []
    pvalues_e = []
    pvalues_i = []
    k_statistics = []
    k_pvalues = []
    man_statistics = []
    man_pvalues = []
    # https://scikit-posthocs.readthedocs.io/en/latest/intro.html
    # parametric or not
    statistic_e, pvalue_e = stats.kstest(data_e, 'norm')
    statistic_i, pvalue_i = stats.kstest(data_i, 'norm')
    statistics_e.append(np.round(statistic_e, 2))
    statistics_i.append(np.round(statistic_i, 2))
    pvalues_e.append(pvalue_e)
    pvalues_i.append(pvalue_i)

    # test not parametric chosen = Kruskal 
    k_statistic, k_pvalue = stats.kruskal(df["E_"+section].tolist(), df["I_"+section].tolist())
    k_statistics.append(np.round(k_statistic, 2))
    k_pvalues.append(np.round(k_pvalue, 2))
    # if pvalues is less than 0.05 -> groups are significantly different
    
    # mannwhitneyu -> more than 0.05 -> same distribution
    man_statistic, man_pvalue = stats.mannwhitneyu(df["E_"+section].tolist(), df["I_"+section].tolist())
    man_statistics.append(np.round(man_statistic, 2))
    man_pvalues.append(np.round(man_pvalue, 2))

    row = statistics[statistics['SECTION'] == section].index
    statistics.loc[row, "e_kstest_stat"] = statistics_e
    statistics.loc[row, "i_kstest_stat"] = statistics_i
    statistics.loc[row, "e_kstest_pvalue"] = pvalues_e
    statistics.loc[row, "i_kstest_pvalue"] = pvalues_i
    statistics.loc[row, "kruskal_stat"] = k_statistics
    statistics.loc[row, "kruskal_pvalue"] = k_pvalues
    statistics.loc[row, "man_stat"] = man_statistics
    statistics.loc[row, "man_pvalue"] = man_pvalues
    
statistics

Kolmogorov-Smirnov test less than 0.05-> the null hypothesis is rejected and there is sufficient evidence to say that the data does not come from a normal distribution (both explicit and implicit)

Since the data are not parametrics -> Kruskal test to check if data groups are significantly different (if p-value < 0.05) -> not for all

Finally, the distribution of data with Mannwhitneyu -> more than 0.05 -> same distribution -> yes

In [ ]:
import seaborn as sns
from statannotations.Annotator import Annotator

df = pd.read_csv("results_godspeed_2.csv")

#create boxplot by group
fig, ax = plt.subplots(1,1,figsize=(14, 5)) 

ax = sns.boxplot(data=df, x="section", y="values", hue="Behavior", flierprops={"marker": "x"})

'''
annotator = Annotator(ax=ax, data=df, x="section", y="values", hue="Behavior", 
                    pairs=[(("Anthropomorphism", "explicit"), ("Anthropomorphism", "implicit")),
(("Animacy", "explicit"), ("Animacy", "implicit")),
(("Likeability", "explicit"), ("Likeability", "implicit")),
(("Intelligence", "explicit"), ("Intelligence", "implicit")),
(("Safety", "explicit"), ("Safety", "implicit")),
(("OVERALL", "explicit"), ("OVERALL", "implicit")),
(("SYSUSE", "explicit"), ("SYSUSE", "implicit")),
(("INFOQUAL", "explicit"), ("INFOQUAL", "implicit")),
(("INTERQUAL", "explicit"), ("INTERQUAL", "implicit"))], order=sections, hue_order=['explicit', 'implicit'])

annotator.configure(test='Mann-Whitney', text_format='star', loc='inside', verbose=2)
annotator.apply_and_annotate()
'''

ax.set_xticklabels(sections_plot)
ax.set_axisbelow(True)
plt.grid(axis='y')
plt.yticks(np.arange(0, 5.5, 0.5))
plt.ylabel("Scores")
plt.xlabel("Section")
plt.title("Distribution of Godspeed scores")
plt.legend(loc='lower right')
plt.savefig(folder_plots+"/boxplots.jpg", bbox_inches='tight')